# Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

## scraping titulares

In [ ]:
news_url = "https://www.lavozdegalicia.es/coruna/"
response = requests.get(news_url)
soup = BeautifulSoup(response.text, 'html.parser')

news_keys = ['title', 'news_image', 'news_href']
    
list_news = [
    {
        news_keys[0]: news.find(class_="a-min-content").get_text(strip = True),
        news_keys[1]: news.find("div").find("img").get("src") if news.find("div").find("img") != None else None,
        news_keys[2]: news_url+news.find("a").get("href")
    }
    for news in soup.find_all(class_="article-min")]
    
list_news


In [ ]:
url_movies = "https://www.sensacine.com/cines/cine/E0770/"
response = requests.get(url_movies)
soup = BeautifulSoup(response.text, 'html.parser')

movie_keys = ['title','synopsis', 'image', 'link_ref']

list_movies = [
    {
        movie_keys[0]: movie.find(class_="meta-title-link").text,
        movie_keys[1]: movie.find(class_="synopsis").text.replace("\n", ""),
        movie_keys[2]: movie.find("img").get("src") if movie.find("img").get("src").startswith("https") else movie.find("img").get("data-src"),
        movie_keys[3]: "https://www.sensacine.com" + movie.find(class_="meta-title-link").get("href")
    }
    for movie in soup.find_all(class_="movie-card-theater")]

list_movies

## LEB ORO 

In [ ]:
url_feb = "https://baloncestoenvivo.feb.es/resultados/ligaleboro/1/2023"
response = requests.get(url_feb)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find(id="_ctl0_MainContentPlaceHolderMaster_jornadaDataGrid")

match_list = []

leb_match_keys = ['teams', 'result', 'date', 'time']

for fila in table.find_all('tr')[1:]:
    temp_dict = {}
    
    for key, linha in zip(leb_match_keys, fila.find_all(['td', 'th'])):
        temp_dict[key] = linha.get_text(strip=True)
    
    match_list.append(temp_dict)

match_list

In [ ]:
import matplotlib.pyplot as plt

ladder = soup.find(id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid")

ladder_list = []

leb_ladder_keys = ['POSITION', 'TEAMS', 'MATCHES_PLAYED', 'MATCHES_W', 'MATCHES_L', 'POINTS']

for row in ladder.find_all('tr')[1:]:
    temp_dict = {}

    cells = row.find_all(['td', 'th'])
    for key, cell in zip(leb_ladder_keys, cells):
        if key == 'POINTS':
            temp_dict[key] = cells[7].get_text(strip=True) 
        else:
            temp_dict[key] = cell.get_text(strip=True)

    ladder_list.append(temp_dict)

df = pd.DataFrame(ladder_list)

fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('off')

col_widths = [0.1, 0.4, 0.2, 0.2, 0.2, 0.2]

table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center', colWidths=col_widths)

plt.tight_layout()
plt.show()
